<a href="https://colab.research.google.com/github/estskyway/study_data_analytics/blob/main/codes/NLP/healthapp_review_LDA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

refer : https://heytech.tistory.com/401

In [ ]:
# !pip install python-mecab-ko pyLDAvis

In [ ]:
import numpy as np

## 전처리

In [ ]:
import pandas as pd
df_review_negative = pd.read_csv('./review_negative.csv')
df_review_negative[:2]

,Unnamed: 0,app,review,rating,tokenized_review,replaced_review
0,1072,매일 운동 - 운동 & 피트니스 트레이너 - 빠르고 효율적인 운동,너무 힘들다,1,NaN,NaN
1,1167,매일 운동 - 운동 & 피트니스 트레이너 - 빠르고 효율적인 운동,훌륭,1,NaN,NaN


In [ ]:
df_review_positive = pd.read_csv('./review_positive.csv')
df_review_positive

,Unnamed: 0,app,review,rating,tokenized_review,replaced_review
0,1000,매일 운동 - 운동 & 피트니스 트레이너 - 빠르고 효율적인 운동,👍,5,NaN,NaN
1,1001,매일 운동 - 운동 & 피트니스 트레이너 - 빠르고 효율적인 운동,힘들지만 살이 빠지는것 같아요. 땀이 많이나요 진짜 말을 할수가 없어...,5,최고,최고
2,1002,매일 운동 - 운동 & 피트니스 트레이너 - 빠르고 효율적인 운동,너무좋아요,5,NaN,NaN
3,1003,매일 운동 - 운동 & 피트니스 트레이너 - 빠르고 효율적인 운동,짱,5,NaN,NaN
4,1004,매일 운동 - 운동 & 피트니스 트레이너 - 빠르고 효율적인 운동,말이 필요없다. 운동 앱중에선 단연 이 어플이 최고!,5,필요 운동 최고,필요 운동 최고
...,...,...,...,...,...,...
7582,8995,30일만에 뱃살 빼기 - 평평한 배 만들기,따라하기도싶고 잼있게 할수있네요..추천입니다,5,추천,추천
7583,8996,30일만에 뱃살 빼기 - 평평한 배 만들기,운동하기 좋고 영상까지 나와 효과적입니다.,5,운동 영상,운동 영상
7584,8997,30일만에 뱃살 빼기 - 평평한 배 만들기,참 좋은앱!ㅎㅎㅎ,5,NaN,NaN
7585,8998,30일만에 뱃살 빼기 - 평평한 배 만들기,쉽고 재미있고 별로 힘들지 않아서 좋아요! 한번 깔아보세요! 추천합니다!! 살 빼고...,5,추천,추천


In [ ]:
df_review_negative = df_review_negative.dropna()
df_review_positive = df_review_positive.dropna()

In [ ]:
df_review_positive.info()

<class 'pandas.core.frame.DataFrame'>
Index: 4351 entries, 1 to 7585
Data columns (total 6 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   Unnamed: 0        4351 non-null   int64 
 1   app               4351 non-null   object
 2   review            4351 non-null   object
 3   rating            4351 non-null   int64 
 4   tokenized_review  4351 non-null   object
 5   replaced_review   4351 non-null   object
dtypes: int64(2), object(4)
memory usage: 237.9+ KB


In [ ]:
df_review_negative.info()

<class 'pandas.core.frame.DataFrame'>
Index: 346 entries, 3 to 412
Data columns (total 6 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   Unnamed: 0        346 non-null    int64 
 1   app               346 non-null    object
 2   review            346 non-null    object
 3   rating            346 non-null    int64 
 4   tokenized_review  346 non-null    object
 5   replaced_review   346 non-null    object
dtypes: int64(2), object(4)
memory usage: 18.9+ KB


## LDA 분석

### negative

In [ ]:
from gensim.corpora import Dictionary

In [ ]:
df_review_negative[['replaced_review']]

,replaced_review
3,칼로리
5,마음
7,오류
8,모야
9,느림
...,...
406,운동 리뷰 출창
407,광고 짜증
408,운동 친절 운동 운동 체계 순서 운동 기분 시작 운동 구간 시작
409,광고 운동


In [ ]:
dictionary = Dictionary(df_review_negative[['replaced_review']].values)

In [ ]:
dictionary.doc2bow(df_review_negative[['replaced_review']].values[3])

[(3, 1)]

In [ ]:
# dictionary에 의한 한글 문장을 백터화 변환
corpus_list = list()
for sentence in df_review_negative[['replaced_review']].values:
  vectors = dictionary.doc2bow(sentence)
  corpus_list.append(vectors)


In [ ]:
df_review_negative['문장백터화'] = corpus_list

In [ ]:
df_review_negative.columns

Index(['Unnamed: 0', 'app', 'review', 'rating', 'tokenized_review',
       'replaced_review', '문장백터화'],
      dtype='object')

#### 토픽 잡기

In [ ]:
from gensim.models import LdaModel

In [ ]:
lda_model = LdaModel(corpus=corpus_list, id2word=dictionary, num_topics=3)

In [ ]:
lda_model.print_topics(num_words=4)

[(0, '0.010*"자동 결제 환불" + 0.010*"최악" + 0.009*"위젯" + 0.009*"걸음"'),
 (1, '0.027*"로그인" + 0.027*"환불" + 0.024*"운동" + 0.014*"불편"'),
 (2, '0.014*"결제 취소" + 0.013*"결제 환불" + 0.009*"운동 겁니다" + 0.009*"자전거 인식 걸음"')]

### positive

In [ ]:
from gensim.corpora import Dictionary

In [ ]:
df_review_positive[['replaced_review']]

,replaced_review
1,최고
4,필요 운동 최고
5,운동 최고
7,집중 운동
12,운동 도움
...,...
7579,리뷰 설명 타임 운동 추천 건강
7580,운동 몸무게 점차 저녁 기대
7582,추천
7583,운동 영상


In [ ]:
pos_dictionary = Dictionary(df_review_positive[['replaced_review']].values)

In [ ]:
pos_dictionary.doc2bow(df_review_positive[['replaced_review']].values[3])

[(3, 1)]

In [ ]:
# dictionary에 의한 한글 문장을 백터화 변환
corpus_list = list()
for sentence_pos in df_review_positive[['replaced_review']].values:
  vectors = dictionary.doc2bow(sentence_pos)
  corpus_list.append(vectors)


In [ ]:
df_review_positive['긍정문장백터화'] = corpus_list

<ipython-input-23-159d16ef79fc>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_review_positive['긍정문장백터화'] = corpus_list


In [ ]:
df_review_positive.columns

Index(['Unnamed: 0', 'app', 'review', 'rating', 'tokenized_review',
       'replaced_review', '긍정문장백터화'],
      dtype='object')

#### 토픽 잡기

In [ ]:
from gensim.models import LdaModel

In [ ]:
lda_model = LdaModel(corpus=corpus_list, id2word=pos_dictionary, num_topics=3)

In [ ]:
lda_model.print_topics(num_words=4)

[(0,
  '0.370*"동영상 동작 설정 구매" + 0.031*"설정 운동 화면 가로 모드" + 0.010*"새해 운동 감사" + 0.006*"운동 방법"'),
 (1,
  '0.188*"동작 추가 기본 운동" + 0.056*"분배 티비" + 0.005*"동영상 동작 설정 구매" + 0.002*"근력 도움"'),
 (2, '0.025*"직접 운동" + 0.017*"성취" + 0.013*"동영상 동작 설정 구매" + 0.013*"무료 운동 체력"')]

### LDA 시각화

In [ ]:
# %pip install pyLDAvis

In [ ]:
import pyLDAvis.gensim_models

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
pyLDAvis.enable_notebook() # 일반 python에선 불필요
result_visualized = pyLDAvis.gensim_models.prepare(lda_model, corpus_list, pos_dictionary)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
pyLDAvis.display(result_visualized)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
pyLDAvis.save_html(result_visualized, './result_visualized.html')

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
